In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class LinearRegression:
    # 实现线性回归算法
    # 指定方法，梯度下降法的学习率，最大迭代次数，收敛阈值
    def __init__(self, method='gradient_descent', learning_rate=0.01, max_iters=1000, tol=1e-4):
        self.method = method
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.tol = tol
        self.theta = None  # 存储类型的参数
        self.mean = None  # 存储数据的均值
        self.std = None  # 存储标准差

    def _add_intercept(self, X):
        """
        为特征矩阵 X 添加截距项
        :param X: 特征矩阵
        :return: 带有截距项的特征矩阵
        """
        intercept = np.ones((X.shape[0], 1))
        return np.hstack((intercept, X))

    def _normalize(self, X):
        """
        对特征矩阵 X 进行标准化处理
        :param X: 特征矩阵
        :return: 标准化后的特征矩阵
        """
        if self.mean is None or self.std is None:
            self.mean = np.mean(X, axis=0)   #计算每列的均值
            self.std = np.std(X, axis=0)     #计算每列的标准差
            self.std[self.std == 0] = 1  # 把标准差为0的替换为1，避免除以零
        return (X - self.mean) / self.std

    def fit(self, X, y):
        """
        训练线性回归模型
        :param X: 特征矩阵
        :param y: 目标向量
        :return: 训练好的模型
        """
        X = self._add_intercept(X)  #添加截距项
        y = y.reshape(-1, 1)   #目标销量y转化为列向量
        m, n = X.shape #获取X的行数和列数

        if self.method == 'least_squares':  #使用最小二乘法
            XTX = X.T.dot(X)
            XTX += np.eye(n) * 1e-8  # 正则化以避免奇异矩阵
            try:
                self.theta = np.linalg.inv(XTX).dot(X.T).dot(y)
            except np.linalg.LinAlgError:
                print("矩阵不可逆，无法使用最小二乘法求解。")
                return self
        elif self.method == 'gradient_descent':   #使用梯度下降法
            X_features = X[:, 1:]  #获取X除去截距项的部分
            X_features = self._normalize(X_features)  #标准化分析
            X = np.hstack((X[:, 0].reshape(-1, 1), X_features))  #标准化后的特征矩阵与截距项拼接
            self.theta = np.zeros((n, 1))  #初始化参数向量为零向量
            prev_loss = float('inf')
            for _ in range(self.max_iters):  #进行最大迭代次数的迭代
                y_pred = X.dot(self.theta)  #计算预测值
                error = y_pred - y  #计算误差值
                loss = np.sum(error ** 2) / (2 * m)  #计算损失函数
                gradient = X.T.dot(error) / m  #计算梯度
                self.theta -= self.learning_rate * gradient  #更新参数向量
                if abs(prev_loss - loss) < self.tol:      
                    break
                prev_loss = loss
        else:
            raise ValueError("Method not supported. Use 'least_squares' or 'gradient_descent'")
        return self

    def predict(self, X):
        """
        使用训练好的模型进行预测
        :param X: 特征矩阵
        :return: 预测结果
        """
        X = self._add_intercept(X)
        if self.method == 'gradient_descent':
            X_features = X[:, 1:]
            X_features = (X_features - self.mean) / self.std  #对特征矩阵进行标准化处理
            X = np.hstack((X[:, 0].reshape(-1, 1), X_features)) #拼接
        return X.dot(self.theta).flatten()  #计算预测值并将其转化为一维数组

    def mse(self, y_true, y_pred):
        """
        计算均方误差
        :param y_true: 真实值
        :param y_pred: 预测值
        :return: 均方误差
        """
        if len(y_true) != len(y_pred):
            raise ValueError("y_true 和 y_pred 的长度必须一致。")
        return np.mean((y_true - y_pred) ** 2)

    def r2_score(self, y_true, y_pred):
        """
        计算 R² 分数
        :param y_true: 真实值
        :param y_pred: 预测值
        :return: R² 分数
        """
        if len(y_true) != len(y_pred):
            raise ValueError("y_true 和 y_pred 的长度必须一致。")
        ss_res = np.sum((y_true - y_pred) ** 2)  #计算残差平方和
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  #计算总平方和
        return 1 - (ss_res / ss_tot) 


# 示例使用
if __name__ == "__main__":
    # 生成示例数据
    np.random.seed(42)
    X = 2 * np.random.rand(100, 1)
    y = 4 + 3 * X + np.random.randn(100, 1).ravel()

    # 划分训练集和测试集
    split = 80
    X_train, y_train = X[:split], y[:split]  #划分训练集
    X_test, y_test = X[split:], y[split:]  #划分测试集

    # 最小二乘法
    model_ls = LinearRegression(method='least_squares')
    model_ls.fit(X_train, y_train)  #训练模型
    y_pred_ls = model_ls.predict(X_test) #进行预测
    print("Least Squares Coefficients:", model_ls.theta.T)
    print("MSE (Least Squares):", model_ls.mse(y_test, y_pred_ls))
    print("R² (Least Squares):", model_ls.r2_score(y_test, y_pred_ls))

    # 梯度下降法
    model_gd = LinearRegression(method='gradient_descent', learning_rate=0.1, max_iters=1000)
    model_gd.fit(X_train, y_train)
    y_pred_gd = model_gd.predict(X_test)
    print("\nGradient Descent Coefficients:", model_gd.theta.T)
    print("MSE (Gradient Descent):", model_gd.mse(y_test, y_pred_gd))
    print("R² (Gradient Descent):", model_gd.r2_score(y_test, y_pred_gd))
    

ValueError: shapes (2,80) and (8000,1) not aligned: 80 (dim 1) != 8000 (dim 0)